# Simple linear model

Try and train a simple linear model for one center, for one forecast date, for one lead time.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import dask
import dask.array as da
import dask.distributed
import dask_jobqueue
import pathlib
import xarray as xr

In [ ]:
INPUT_TRAIN = '***HOME***hdd_scratch/s2s/training-input/0.3.0/netcdf'
input_path = pathlib.Path(INPUT_TRAIN)

In [ ]:
OUTPUT_TRAIN = '***HOME***hdd_scratch/s2s/training-output-benchmark/'

In [ ]:
REFERENCE_TRAIN = '***HOME***hdd_scratch/s2s/training-output-reference/'

## Boot Dask cluster

In [ ]:
cluster = dask_jobqueue.SLURMCluster(
    env_extra=['source ***HOME***.bash_profile','conda activate s2s'],
)

In [ ]:
cluster.scale(jobs=2)  # Scale to two working nodes as configured.
client = dask.distributed.Client(cluster)

In [ ]:
client

## Read input data

In [ ]:
CENTER = 'ecmwf'
DATE = '20200102'

In [ ]:
files = [f for f in input_path.iterdir() if CENTER in f.stem if DATE in f.stem]

In [ ]:
files

In [ ]:
variable = [x.stem.split('-')[2] for x in files]

In [ ]:
sorted(variable)

In [ ]:
'entireAtmosphere' in datasets[0].coords

In [ ]:
datasets[0]

In [ ]:
dataset_with_level = []
dataset_wo_level = []

for d in datasets:
    if 'plev' in d.coords:
        dataset_with_level.append(d)
    else:
        dataset_wo_level.append(d)

In [ ]:
dataset_wo_level[0]

In [ ]:
full_dataset = xr.merge(dataset_wo_level)

In [ ]:
full_dataset

In [ ]:
one_lead_time = full_dataset.isel(lead_time=21, meanSea=0, entireAtmosphere=0, nominal_top=0).drop(['rsn', 'siconc', 'lsm', 'sst', 'tp'])

In [ ]:
one_lead_time

In [ ]:
one_lead_time.isnull().sum()

In [ ]:
one_lead_time.shape

In [ ]:
np_input = one_lead_time.to_array(dim='field').transpose('forecast_time', 'latitude', 'longitude', 'field', 'realization').data

In [ ]:
np_input = np_input.reshape(20, 121, 240, -1)

In [ ]:
np_input.shape

## Reading output values

In [ ]:
!ls {OUTPUT_TRAIN}

In [ ]:
t2m = xr.open_dataset(OUTPUT_TRAIN + '/t2m.nc')

In [ ]:
t2m

In [ ]:
first_forecast_t2m = t2m.sel(forecast_time=t2m.forecast_time.dt.day == 2).isel(lead_time=0)

In [ ]:
first_forecast_t2m

In [ ]:
t2m.lead_time.data.astype('timedelta64[D]')

In [ ]:
first_forecast_t2m.isel(category=2, forecast_time=20).t2m.plot()

## Check one of the bi-weekly files

In [ ]:
t2m_weekly = xr.open_dataset(OUTPUT_TRAIN + '/t2m-weeks-34.nc')

In [ ]:
t2m_weekly

## Check the other type of forecast

In [ ]:
reference_t2m = xr.open_dataset(REFERENCE_TRAIN + '/t2m-20200102.nc')

In [ ]:
reference_t2m

In [ ]:
reference_t2m.lead_time.data.astype('timedelta64[D]')